In [1]:
from datasets import load_dataset
import jsonlines

from pprint import pprint 

In [2]:
finetuned_dataset = load_dataset("lamini/lamini_docs", split="train")
finetuned_dataset 

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1260
})

In [4]:
# finetuned_dataset[0]

In [3]:
# for i in range(3):
#     pprint(finetuned_dataset[i])

### Revise map, filter and reduce

In [15]:
vee_list = [3, 4, 5]

add_list = []

for item in vee_list:
    if item > 4:
        add_list.append(item)
    
add_list

[5]

In [14]:
tuple(map(lambda x: x + 1, vee_list))

(4, 5, 6)

In [16]:
list(filter(lambda x: x > 4, vee_list))

[5]

In [19]:
from functools import reduce

reduce(lambda x, y: x * y, vee_list)

60

### Assignment: Create `lamini_docs.jsonl`

In [8]:
new_list = list(
    map(
        lambda lamini_dict: {
            "question": lamini_dict["question"],
            "answer": lamini_dict["answer"],
        },
        finetuned_dataset,
    )
)

assert len(new_list) == len(finetuned_dataset)

pprint(new_list[0])

{'answer': 'There are several metrics that can be used to evaluate the '
           'performance and quality of generated text from Lamini models, '
           'including perplexity, BLEU score, and human evaluation. Perplexity '
           'measures how well the model predicts the next word in a sequence, '
           'while BLEU score measures the similarity between the generated '
           'text and a reference text. Human evaluation involves having human '
           'judges rate the quality of the generated text based on factors '
           'such as coherence, fluency, and relevance. It is recommended to '
           'use a combination of these metrics for a comprehensive evaluation '
           "of the model's performance.",
 'question': 'How can I evaluate the performance and quality of the generated '
             'text from Lamini models?'}


In [22]:
with jsonlines.open(f"json/lamini_docs.jsonl", "w") as writer:
    writer.write_all(new_list)

### Compare pre-training and fine-tuning dataset

In [5]:
import itertools
import pandas as pd

In [6]:
pretrained_dataset = load_dataset("c4", "en", split="train", streaming=True)

In [7]:
n = 2
print("Pretrained dataset:")
top_n = itertools.islice(pretrained_dataset, n)
for i in top_n:
    pprint(i)

Pretrained dataset:
{'text': 'Beginners BBQ Class Taking Place in Missoula!\n'
         'Do you want to get better at making delicious BBQ? You will have the '
         'opportunity, put this on your calendar now. Thursday, September 22nd '
         'join World Class BBQ Champion, Tony Balay from Lonestar Smoke '
         'Rangers. He will be teaching a beginner level class for everyone who '
         'wants to get better with their culinary skills.\n'
         'He will teach you everything you need to know to compete in a KCBS '
         'BBQ competition, including techniques, recipes, timelines, meat '
         'selection and trimming, plus smoker and fire information.\n'
         'The cost to be in the class is $35 per person, and for spectators it '
         'is free. Included in the cost will be either a t-shirt or apron and '
         'you will be tasting samples of each meat that is prepared.',
 'timestamp': '2019-04-25T12:57:54Z',
 'url': 'https://klyq.com/beginners-bbq-class-t

In [26]:
filename = "json/lamini_docs.jsonl"
instruction_dataset_df = pd.read_json(filename, lines=True)
instruction_dataset_df

,question,answer
0,How can I evaluate the performance and quality...,There are several metrics that can be used to ...
1,Can I find information about the code's approa...,"Yes, the code includes methods for submitting ..."
2,How does Lamini AI handle requests for generat...,Lamini AI offers features for generating text ...
3,Does the `submit_job()` function expose any ad...,It is unclear which `submit_job()` function is...
4,Does the `add_data()` function support differe...,"No, the `add_data()` function does not support..."
...,...,...
1255,Does the documentation provide guidelines for ...,There is no mention of memory caching or evict...
1256,Does Lamini provide any mechanisms for model e...,"Yes, Lamini provides mechanisms for model ense..."
1257,Is Lamini owned by Tesla?,"No, Lamini AI is an independent company workin..."
1258,What is the process for suggesting edits or im...,You can suggest edits or improvements to the L...


In [28]:
examples = instruction_dataset_df.to_dict()
text = examples["question"][0] + examples["answer"][0]
pprint(text)

('How can I evaluate the performance and quality of the generated text from '
 'Lamini models?There are several metrics that can be used to evaluate the '
 'performance and quality of generated text from Lamini models, including '
 'perplexity, BLEU score, and human evaluation. Perplexity measures how well '
 'the model predicts the next word in a sequence, while BLEU score measures '
 'the similarity between the generated text and a reference text. Human '
 'evaluation involves having human judges rate the quality of the generated '
 'text based on factors such as coherence, fluency, and relevance. It is '
 'recommended to use a combination of these metrics for a comprehensive '
 "evaluation of the model's performance.")


In [32]:
word = "{name} is very fond of visiting {place}"

word.format(name="Vee", place="Calabar")

'Vee is very fond of visiting Calabar'

In [33]:
name, place = "Vee", "Calabar"

f"{name} is very fond of visiting {place}"

'Vee is very fond of visiting Calabar'

In [29]:
prompt_template_qa = """### Question:
{question}

### Answer:
{answer}"""

In [35]:
question = examples["question"][0]
answer = examples["answer"][0]

text_with_prompt_template = prompt_template_qa.format(question=question, answer=answer)
pprint(text_with_prompt_template)

('### Question:\n'
 'How can I evaluate the performance and quality of the generated text from '
 'Lamini models?\n'
 '\n'
 '### Answer:\n'
 'There are several metrics that can be used to evaluate the performance and '
 'quality of generated text from Lamini models, including perplexity, BLEU '
 'score, and human evaluation. Perplexity measures how well the model predicts '
 'the next word in a sequence, while BLEU score measures the similarity '
 'between the generated text and a reference text. Human evaluation involves '
 'having human judges rate the quality of the generated text based on factors '
 'such as coherence, fluency, and relevance. It is recommended to use a '
 "combination of these metrics for a comprehensive evaluation of the model's "
 'performance.')


In [36]:
prompt_template_q = """### Question:
{question}

### Answer:"""

In [37]:
num_examples = len(examples["question"])
finetuning_dataset_text_only = []
finetuning_dataset_question_answer = []

for i in range(num_examples):
    question = examples["question"][i]
    answer = examples["answer"][i]

    text_with_prompt_template_qa = prompt_template_qa.format(
        question=question, answer=answer
    )
    finetuning_dataset_text_only.append({"text": text_with_prompt_template_qa})

    text_with_prompt_template_q = prompt_template_q.format(question=question)
    finetuning_dataset_question_answer.append(
        {"question": text_with_prompt_template_q, "answer": answer}
    )

In [38]:
pprint(finetuning_dataset_text_only[0])

{'text': '### Question:\n'
         'How can I evaluate the performance and quality of the generated text '
         'from Lamini models?\n'
         '\n'
         '### Answer:\n'
         'There are several metrics that can be used to evaluate the '
         'performance and quality of generated text from Lamini models, '
         'including perplexity, BLEU score, and human evaluation. Perplexity '
         'measures how well the model predicts the next word in a sequence, '
         'while BLEU score measures the similarity between the generated text '
         'and a reference text. Human evaluation involves having human judges '
         'rate the quality of the generated text based on factors such as '
         'coherence, fluency, and relevance. It is recommended to use a '
         'combination of these metrics for a comprehensive evaluation of the '
         "model's performance."}


In [39]:
pprint(finetuning_dataset_question_answer[0])

{'answer': 'There are several metrics that can be used to evaluate the '
           'performance and quality of generated text from Lamini models, '
           'including perplexity, BLEU score, and human evaluation. Perplexity '
           'measures how well the model predicts the next word in a sequence, '
           'while BLEU score measures the similarity between the generated '
           'text and a reference text. Human evaluation involves having human '
           'judges rate the quality of the generated text based on factors '
           'such as coherence, fluency, and relevance. It is recommended to '
           'use a combination of these metrics for a comprehensive evaluation '
           "of the model's performance.",
 'question': '### Question:\n'
             'How can I evaluate the performance and quality of the generated '
             'text from Lamini models?\n'
             '\n'
             '### Answer:'}


In [40]:
with jsonlines.open(f"json/lamini_docs_processed.jsonl", "w") as writer:
    writer.write_all(finetuning_dataset_question_answer)